In [ ]:
import os 
import json
import pandas as pd
import traceback

In [ ]:
from langchain_community.chat_models import ChatOpenAI


In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = ChatOpenAI(openai_api_key = KEY , model = "openai/gpt-4.1", base_url= "https://models.github.ai/inference", temperature = 0.5)

In [ ]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain , SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [ ]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct" : "correct answer", 
    },
    "2": {
        "mcq": "multiple choice question",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct" : "correct answer", 
    },
    "3": {
        "mcq": "multiple choice question",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct" : "correct answer", 
    },
}

In [ ]:
TEMPLATE = """
Text = {text}
you are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}


"""

In [ ]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [ ]:
quiz_chain= LLMChain(llm= llm, prompt = quiz_generation_prompt, output_key= "quiz", verbose = True)

In [ ]:
TEMPLATE2 = """
You are an expert english gramarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You nedd to evaluate the complexity of the questions ad give a complete analysis of the quiz. Only use at max 50 words for complexity,
if the quiz is not at par with the cognitive and analytical abilities of the student,\
update the quiz questions which needs to be chnaged and change the tone such that it perfectly fits the student abilities.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz: 
"""

In [ ]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template= TEMPLATE2)

In [ ]:
review_chain = LLMChain(llm = llm , prompt = quiz_evaluation_prompt, output_key= "review", verbose= True)

In [ ]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [ ]:
file_path = r"/home/nihar/Projects/GenAi/mcqgen/data.txt"

In [ ]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [ ]:
print(TEXT)

In [ ]:
json.dumps(RESPONSE_JSON)

In [ ]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [ ]:
#to track the token usage 

with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text" :TEXT,
            "number" :NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

In [ ]:
response

In [ ]:
quiz=response.get("quiz")

In [ ]:
quiz =json.loads(quiz)

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
quiz = pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv",index=False)